In [11]:
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import wandb

In [12]:
# Initialize WandB
wandb.init(project="assignment1", entity="da6401-assignments")

In [ ]:
# Define hyperparameters
config = wandb.config
config.hidden_layers = [256,128, 64]  # Two hidden layers with 128 and 64 neurons
config.input_size = 784  # 28x28 images
config.output_size = 10  # 10 classes for Fashion-MNIST

In [14]:
# Load the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train.reshape(-1, config.input_size) / 255.0
x_test = x_test.reshape(-1, config.input_size) / 255.0

In [15]:
class FeedforwardNeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size):
        """
        Initialize the neural network.
        :param input_size: Number of input features (e.g., 784 for 28x28 images).
        :param hidden_layers: List containing the number of neurons in each hidden layer.
        :param output_size: Number of output classes (e.g., 10 for Fashion-MNIST).
        """
        self.layers = []
        self.weights = []
        self.biases = []

        # Input layer to first hidden layer
        prev_size = input_size
        for layer_size in hidden_layers:
            self.weights.append(np.random.randn(prev_size, layer_size) * 0.01)
            self.biases.append(np.zeros((1, layer_size)))
            prev_size = layer_size

        # Hidden layers to output layer
        self.weights.append(np.random.randn(prev_size, output_size) * 0.01)
        self.biases.append(np.zeros((1, output_size)))

    def sigmoid(self, x):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        """Softmax activation function."""
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, x):
        """
        Perform a forward pass through the network.
        :param x: Input data.
        :return: Output probabilities.
        """
        self.layers = [x]
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            x = self.sigmoid(np.dot(x, w) + b)
            self.layers.append(x)

        # Output layer with softmax
        output = self.softmax(np.dot(x, self.weights[-1]) + self.biases[-1])
        self.layers.append(output)
        return output

In [17]:
network = FeedforwardNeuralNetwork(config.input_size, config.hidden_layers, config.output_size)
pred_probs = network.forward(x_train)
print(pred_probs[0])
wandb.finish()

[0.10005263 0.10220131 0.10245415 0.10139055 0.10543052 0.10293845
 0.09735915 0.10225377 0.09164533 0.09427413]
